In [2]:
import seaborn as sns, pandas as pd, numpy as np, yfinance as yf, matplotlib.pyplot as plt
from scipy import stats
from pandas.tseries.offsets import BMonthEnd, BusinessDay
from datetime import date

#calculate R^2
def slope(ts):
    x = np.arange(len(ts))
    log_ts = np.log(ts)
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    score = annualized_slope * (r_value ** 2)
    return score

#Inverse Volatility Weight portfolio consturction
def inv_vola_calc(ts):
    vola_window = 20
    return ts.pct_change().rolling(vola_window).std().dropna().iloc[-1]

#Setting Params for Momentum
def get_params():
    minimum_momentum = 0  # momentum score cap
    number_of_stocks = 27
    exclude_days = 0
    return minimum_momentum, number_of_stocks, exclude_days
def build_mom_list(momentum_window, tickers_list, total_hist):
    minimum_momentum, number_of_stocks, exclude_days = get_params()

    hist = pd.DataFrame(columns=tickers_list)

    hist_window = momentum_window + exclude_days
    for ticker in tickers_list:
        ticker = str(ticker)
        hist[ticker] = total_hist[ticker]["Close"].tail(hist_window)
        # volume[ticker] = total_hist[ticker]["Volume"].tail(200+2)[:-2]

    data_end = -1 * (exclude_days + 1)  # exclude most recent data

    hist = hist.dropna()

    momentum1_start = -1 * (momentum_window + exclude_days)
    momentum_hist1 = hist[momentum1_start:data_end]

    momentum_list = momentum_hist1.apply(slope)  # Mom Window 1

    ranking_table = momentum_list.sort_values(ascending=False)

    ranking_table.to_csv("sector_ranking_table.csv")
    return ranking_table, hist
def analyze(total_hist):
    d=date.today()
    idx = pd.IndexSlice
    year_start = total_hist.loc['2021-12-31',idx[:,'Close']]

    offset = BusinessDay(n=0)
    curr_bd = (d - offset).strftime('%Y-%m-%d')
    current = total_hist.loc[curr_bd,idx[:,'Close']]

    ytd_df = pd.merge(year_start,current, right_index = True, left_index = True)
    ytd_df['YTD Return %'] = (ytd_df[curr_bd]- ytd_df['2021-12-31']) / ytd_df['2021-12-31']

    offset = BMonthEnd()

    prev_mtd = offset.rollback(d).strftime('%Y-%m-%d')
    prev_mtd_srs = total_hist.loc[prev_mtd,idx[:,'Close']]

    mtd_df = pd.merge(prev_mtd_srs,current, right_index = True, left_index = True)
    mtd_df['MTD Return %'] = (mtd_df[curr_bd]- mtd_df[prev_mtd]) / mtd_df[prev_mtd]

    returns_df = pd.DataFrame(index=mtd_df.index)

    returns_df['MTD Return %'],returns_df['YTD Return %'] = mtd_df['MTD Return %'],ytd_df['YTD Return %']

    returns_df = returns_df.droplevel(level=1).sort_values(by = 'YTD Return %', ascending = False)

    returns_df.to_csv("returns_df_Comm.csv")

    dict_cols = {20: '1-Month',
                 60: '3-Months',
                 125: '6-Months',
                 '20-corr': '1-Month Corr',
                 '60-corr': '3-Months Corr',
                 '125-corr': '6-Months Corr'}

    time_list = [20, 60, 125]
    r2_table = pd.DataFrame(columns=time_list, index=tickers_list).sort_index()

    for time in time_list:
        ranking_t, hist_for_corr = build_mom_list(time, tickers_list, total_hist)

        r2_ranks = pd.DataFrame(ranking_t).sort_index()
        r2_table[time] = r2_ranks[0] / (125 / time)

        r2_table[str(time) + '-corr'] = np.nan

        for tick in tickers_list:
            r2_table.loc[tick, str(time) + '-corr'] = hist_for_corr.corr()[tick].drop(tick).mean()

    r2_table.insert(3, 'Avg Alpha-Factor', r2_table[time_list].mean(axis=1))
    r2_table = r2_table.rename(columns=dict_cols)
    r2_table = r2_table.sort_values(by='Avg Alpha-Factor', ascending=False)

    vola_table = hist_for_corr.apply(inv_vola_calc)

    r2_table['1-Month Vol'] = vola_table
    r2_table.to_csv("r2_table_Comm.csv")

    pd.set_option('display.max_columns', 1000000)
    pd.set_option('display.max_rows', 1000000)

    plt.rcParams.update({'figure.max_open_warning': 0})

    return r2_table

In [3]:
dfSectors = pd.read_excel("Comm_Universe.xlsx", engine='openpyxl')
tickers_list = dfSectors["Symbol"].values.tolist()
total_hist = yf.download(tickers=tickers_list, period="1y",
                        interval="1d", group_by='ticker',
                        auto_adjust=True, prepost=True,
                        threads=True, proxy=None)

r2_table = analyze(total_hist)

[*********************100%***********************]  26 of 26 completed


In [4]:
r2_table

,1-Month,3-Months,6-Months,Avg Alpha-Factor,1-Month Corr,3-Months Corr,6-Months Corr,1-Month Vol
UYM,0.333237,249.365341,3.439886,84.379488,0.372827,0.529664,0.466984,0.028879
SLV,86.918243,50.576832,4.991841,47.495639,0.341084,0.510322,0.373555,0.021333
GDX,20.425402,85.910804,0.001426,35.445877,0.428721,0.532612,0.357458,0.023833
XME,0.466031,66.368142,10.983776,25.939316,0.353447,0.515975,0.425008,0.022003
PLTM,4.755738,47.138369,15.373490,22.422532,0.441295,0.531147,0.357732,0.017176
PPLT,4.356015,46.529299,15.282230,22.055848,0.440228,0.531669,0.356653,0.017467
CGW,1.152470,63.128903,0.127545,21.469639,0.404506,0.511871,0.403050,0.013276
CUT,4.265289,51.659052,-0.452384,18.490652,0.476336,0.522006,0.358315,0.011737
WOOD,0.253095,48.306254,-0.746587,15.937587,0.403617,0.522557,0.339666,0.012089
PHO,1.421883,43.234501,2.352112,15.669499,0.460393,0.518167,0.387172,0.013825
